# IQM Garnet Native Gates

In this notebook, we will explore the functionality of the native gates of IQM Garnet. Programming a quantum circuit directly with native gates grants direct control over quantum operations on the quantum computer. The circuit will run exactly as you compose it. From this notebook, you will learn the mathematical representations of the native gates of IQM Garnet and how to program quantum circuits with these native gates. 

The native gates of IQM Garnet are the PRx gate and the CZ gate. These gates are physically implemented by applying microwave pulses on the transmon qubits. The PRx gate on its own forms a universal gate set for one-qubit operations. Any one-qubit unitary can be implemented using a few PRx gates. The PRx gate and CZ gate together constitute a universal gate set for quantum computation. Any quantum circuit can be realized using only these native gates.

Let's first import the necessary modules and define a constant.

In [1]:
import numpy as np
import sympy as sp
from sympy import I, nsimplify

from braket.circuits import Circuit
from braket.aws import AwsDevice

pi = np.pi

## Single-qubit native gates

### PRx gate

The PRx gate, or the Phased Rx gate, is a generalized Rx rotation. With Rx($\theta$) gate, you can rotate a one-qubit state around the x-axis by $\theta$. The PRx gate generalizes this. With the PRx gate, you can rotate the one-qubit state around any axis in the x-y plane. The PRx gate has two parameters. The first parameter, $\theta$, defines the angle of rotation around the axis. The second parameter, $\phi$, defines the orientation of the axis in the x-y plane. For example, to rotate around the y-axis, we set $\phi=\pi/2$. To rotate around the x-axis, we set $\phi=0$. 

Mathematically, the PRx gate can be represented as 
$$
PRx(\theta, \phi) = Rz(\phi)Rx(\theta)Rz(-\phi)
$$
You can understand the Rz gates as rotations of the reference frame. The reference frame is rotated by $-\phi$ before applying the Rx($\theta$), followed by rotating the reference frame back to the original one.

In [2]:
rx_symbolic = lambda x: sp.Matrix([[sp.cos(x/2), -1j*sp.sin(x/2)], [-1j*sp.sin(x/2), sp.cos(x/2)]])
rz_symbolic = lambda x: sp.Matrix([[sp.exp(-1j*x/2), 0], [0, sp.exp(1j*x/2)]])
prx_symbolic = lambda a1, a2: rz_symbolic(a2) * rx_symbolic(a1) * rz_symbolic(-a2)

The matrix representation of the PRx gate is

In [3]:
theta, phi = sp.symbols('theta phi')
prx_matrix = prx_symbolic(theta, phi)

nsimplify(prx_matrix, tolerance=1e-8)

Matrix([
[              cos(theta/2), -I*exp(-I*phi)*sin(theta/2)],
[-I*exp(I*phi)*sin(theta/2),                cos(theta/2)]])

Special case when $\phi=0$: the gate PRx($\theta$,$0$) is equivalent to Rx($\theta$). 

In [4]:
theta, phi = sp.symbols('theta phi')
rx_matrix = prx_symbolic(theta, 0)

nsimplify(rx_matrix.evalf(), tolerance=1e-8)

Matrix([
[   cos(theta/2), -I*sin(theta/2)],
[-I*sin(theta/2),    cos(theta/2)]])

Special case when $\phi=\pi/2$: the gate PRx($\theta$,$\pi/2$), is equivalent to Ry($\theta$). 

In [5]:
theta, phi = sp.symbols('theta phi')
ry_matrix = prx_symbolic(theta, pi/2)

nsimplify(ry_matrix.evalf(), tolerance=1e-8)

Matrix([
[cos(theta/2), -sin(theta/2)],
[sin(theta/2),  cos(theta/2)]])

## Two-qubit entangling gates
### CZ gate
The two-qubit native gate of IQM Garnet is the CZ gate. It is a symmetric gate that adds a phase to the portion of the two-qubit quantum state where both qubits are in the |1> state. The matrix representation of the CZ gate is 


In [6]:
cz_symbolic = sp.diag(1, 1, 1, -1)
cz_symbolic

Matrix([
[1, 0, 0,  0],
[0, 1, 0,  0],
[0, 0, 1,  0],
[0, 0, 0, -1]])

## Submitting verbatim circuits to IQM Garnet

Below is an example of executing a circuit on IQM Garnet. When the circuit is submitted to Amazon Braket, it is compiled into an equivalent circuit that only includes native gates, PRx and CZ, before running on the quantum computer.  

In [8]:
device = AwsDevice('arn:aws:braket:eu-north-1::device/qpu/iqm/Garnet')

circuit = Circuit().h(0).cnot(0, 1)

task = device.run(circuit, shots=100)

In certain use cases, you may want to program your circuit and skip the compilation step. For example, in compilation research, you could be experimenting with various ways of preparing a quantum state. In algorithm research, you could be experimenting with error mitigation methods that requires applying gates and their inverses together, and you want to prevent a compiler from removing these gate-inverse pairs. In these cases, you can use verbatim compilation by adding your gates into a verbatim box. Gates enclosed by a verbatim box are not modified by the compiler. They will run exactly as you specified in the quantum circuit.

Several rules apply for operations inside verbatim boxes:
- All qubits must be physical qubits of the device
- All two-qubit gates must be applied to physical edges of the device
- All gates must be native gates of the device

[This notebook](./Verbatim_Compilation.ipynb) shows you how to check the relevant device properties for verbatim compilation. The physical qubits of IQM Garnet are

In [9]:
sorted(device.topology_graph.nodes)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

The physical edges of IQM Garnet are

In [10]:
sorted(device.topology_graph.edges)

[(1, 2),
 (1, 4),
 (2, 5),
 (3, 4),
 (3, 8),
 (4, 5),
 (4, 9),
 (5, 6),
 (6, 7),
 (8, 9),
 (10, 5),
 (10, 9),
 (10, 11),
 (10, 15),
 (11, 6),
 (11, 12),
 (11, 16),
 (12, 7),
 (12, 17),
 (13, 8),
 (13, 14),
 (14, 9),
 (14, 15),
 (14, 18),
 (15, 16),
 (15, 19),
 (16, 17),
 (16, 20),
 (18, 19),
 (19, 20)]

Based on these device properties, we can create a verbatim circuit:

In [11]:
circuit = Circuit().prx(1, pi/2, 1).cz(1, 2)
circuit = Circuit().add_verbatim_box(circuit)
print(circuit)

T  : │        0        │       1        │  2  │       3       │
                        ┌──────────────┐                       
q1 : ───StartVerbatim───┤ PRx(1.57, 1) ├───●─────EndVerbatim───
              ║         └──────────────┘   │          ║        
              ║                          ┌─┴─┐        ║        
q2 : ─────────╨──────────────────────────┤ Z ├────────╨────────
                                         └───┘                 
T  : │        0        │       1        │  2  │       3       │


## A tip for composing verbatim circuits on IQM device - Rz virtualization

Although PRx alone can compose any one-qubit gate, sometime it is more intuitive to write circuits using Rz gates. This section provides a tip that allows you to start writing your circuits with IQM native gates plus Rz gates, and then remove the Rz gates with a process called Rz virtualization. 

Rz virtualization is based on the following two identities:
1. CZ() Rz($\alpha$) = Rz($\alpha$) CZ()
2. PRx($\theta$, $\phi$) Rz($\alpha$) = Rz($\alpha$) PRx($\theta$, $\phi-\alpha$)

To understand where the second identity comes from, let's substitute PRx on the left-hand side with the identity introduced in the beginning of this notebook, PRx($\theta$, $\phi$) = Rz($\phi$) Rx($\theta$) Rz($-\phi$). Following the derivation below, the right-hand side of the identity is obtained:

Rz($\phi$) Rx($\theta$) Rz($-\phi$) Rz($\alpha$)

= Rz($\phi$) Rx($\theta$) Rz($-\phi+\alpha$)

= Rz($\alpha$) Rz($\phi-\alpha$) Rx($\theta$) Rz($-\phi+\alpha$)

= Rz($\alpha$) PRx($\theta$, $\phi-\alpha$)

When you start with a circuit that has only Rz gates in addition to the IQM native gates, you can use these two identities to move all Rz gates to the very end of the circuit, just before the measurement.

In [12]:
circuit_1 = Circuit().rz(1, 0.5).rz(2, 0.7).prx(1, 0.5, 1.5).cz(1, 2)
print(circuit_1)

T  : │     0      │         1         │  2  │
      ┌──────────┐ ┌─────────────────┐       
q1 : ─┤ Rz(0.50) ├─┤ PRx(0.50, 1.50) ├───●───
      └──────────┘ └─────────────────┘   │   
      ┌──────────┐                     ┌─┴─┐ 
q2 : ─┤ Rz(0.70) ├─────────────────────┤ Z ├─
      └──────────┘                     └───┘ 
T  : │     0      │         1         │  2  │


The circuit above is equivalent to the following circuit with Rz gates moved to the end, up to a global phase.

In [13]:
circuit_2 = Circuit().prx(1, 0.5, 1.5-0.5).cz(1, 2).rz(1, 0.5).rz(2, 0.7)
print(circuit_2)

T  : │         0         │  1  │     2      │
      ┌─────────────────┐       ┌──────────┐ 
q1 : ─┤ PRx(0.50, 1.00) ├───●───┤ Rz(0.50) ├─
      └─────────────────┘   │   └──────────┘ 
                          ┌─┴─┐ ┌──────────┐ 
q2 : ─────────────────────┤ Z ├─┤ Rz(0.70) ├─
                          └───┘ └──────────┘ 
T  : │         0         │  1  │     2      │


We can compare the unitaries to make sure the two circuits are indeed equivalent. 

In [15]:
u1 = circuit_1.to_unitary()
u2 = circuit_2.to_unitary() 

product = u1 @ np.linalg.inv(u2)
np.isclose(np.abs(sum(np.diag(product))), 4.0)

True

Finally, because an Rz gate at the end of a circuit does not have any direct effect on the measurement in the z-basis, the measurement results will be the same even if we remove all of these Rz gates that are just prior to the measurement. (Note that you can only remove the Rz gates in this way when your desired output is based on measurement counts or probability distribution of measurement outcomes. If your desired output is a state vector, removing the Rz gates would change the output.) Removing the Rz gates produces the following circuit, which consists only of IQM native gates.

In [16]:
circuit_3 = Circuit().prx(1, 0.5, 1.0).cz(1, 2)
print(circuit_3)

T  : │         0         │  1  │
      ┌─────────────────┐       
q1 : ─┤ PRx(0.50, 1.00) ├───●───
      └─────────────────┘   │   
                          ┌─┴─┐ 
q2 : ─────────────────────┤ Z ├─
                          └───┘ 
T  : │         0         │  1  │


## Summary

This notebook introduces the native gate set of IQM Garnet and demonstrates how to write common gates like Rx and Ry gates in terms of the native PRx gate. An example is presented for composing and submitting verbatim circuits to IQM Garnet. Finally, the notebook provides a tip to help you compose verbatim circuits for IQM Garnet more easily by building circuits with Rz gates and then eliminating them from the circuit using Rz virtualization.  